In [2]:
import pandas as pd

In [18]:
df = pd.read_csv("output3.csv",index_col=0)
df = df[~df["mean"].duplicated()]
df["recent"] = df.index.str.slice(-1)
df = df[df["count"] > 30]
df.head(3)

,win_ratio,mean,count,recent
~speedup1-~opentop0-top0-~yz1-top1-opentop1-4,0.650000,1.020381,40,4
jump1-yz0-~opentop1-top1-3,0.617978,1.021005,89,3
speedup2-~yz0-top0-opentop0-~top1-1,0.617647,1.020963,34,1


In [55]:
sub_df = df[df.index.isin(valid_keys) == True]
sub_df.head(10)

,win_ratio,mean,count,recent
~opentop2-small_volume-yz1-2,0.657143,1.026360,35,2
~opentop2-speedup1-small_volume-yz1-2,0.657143,1.026462,35,2
~opentop2-jump1-small_volume-yz1-2,0.625000,1.023453,32,2
~opentop2-~speedup2-small_volume-yz1-2,0.676471,1.028356,34,2
~opentop2-~foot0-small_volume-yz1-2,0.612903,1.023296,31,2
speedup2-~foot0-small_capq-~yz1-top1-opentop1-2,0.705882,1.022490,34,2
speedup2-small_capq-small_volume-~yz1-top1-opentop1-2,0.666667,1.022768,39,2
~opentop2-jump1-~speedup2-small_volume-yz1-2,0.645161,1.025663,31,2
~opentop2-jump1-~foot2-small_volume-~yz1-top1-opentop1-2,0.622222,1.020907,45,2
~opentop2-~foot0-~foot2-small_volume-~yz1-top1-opentop1-2,0.625000,1.020322,56,2


In [78]:
keys = ["~opentop2-small_volume-yz1-2","~opentop2-speedup1-small_volume-yz1-2","~opentop2-jump1-small_volume-yz1-2"]
valid_keysB = remove_invalid(df[df.index.isin(keys) == True],i)
valid_keysB

~opentop2-small_volume-yz1-2--~opentop2-speedup1-small_volume-yz1-2
~opentop2-small_volume-yz1-2
~opentop2-speedup1-small_volume-yz1-2--~opentop2-jump1-small_volume-yz1-2


{'~opentop2-speedup1-small_volume-yz1-2'}

In [52]:
for i in range(2,3):
    sub_df = df[df["recent"] == str(i)]
    print len(sub_df),i
    valid_keys = remove_invalid(sub_df,i)
    print valid_keys

108 2
set(['speedup2-jump2-~foot0-~foot1-small_capq-~yz1-top1-opentop1-2', '~opentop2-jump1-small_volume-yz1-2', '~opentop2-~speedup2-small_volume-yz1-2', '~speedup1-~jump1-~speedup2-foot1-isnew-minute-2', 'speedup2-small_capq-small_volume-~yz1-top1-opentop1-2', '~opentop2-speedup1-jump1-~speedup2-small_volume-yz1-2', '~opentop2-small_volume-yz1-2', '~opentop2-speedup1-small_volume-yz1-2', 'speedup2-jump2-small_capq-minute-small_volume-~yz1-top1-opentop1-2', 'jump2-~foot0-~foot1-~foot2-small_capq-small_volume-~yz1-top1-opentop1-2', '~opentop2-jump1-~foot2-small_volume-~yz1-top1-opentop1-2', '~speedup1-~jump1-speedup2-foot0-foot1-~isnew-~small_capq-2', 'speedup2-~foot0-small_capq-minute-~yz1-top1-opentop1-2', 'speedup2-jump2-~foot0-~foot1-minute-small_volume-~yz1-top1-opentop1-2', 'speedup2-~foot0-~foot1-small_capq-~yz1-top1-opentop1-2', '~opentop2-speedup1-~jump1-speedup2-jump2-foot0-small_volume-2', 'speedup2-~foot0-small_capq-~yz1-top1-opentop1-2', 'speedup2-jump2-~foot0-small_capq-s

In [77]:
def remove_invalid(output_df,day):
    keys = output_df.index
    valid_keys = set(keys)
    for keyA in keys:
        if not keyA in valid_keys:
            continue
        filterA = getFilter(keyA)
        filter_recent = recent(day,data_df)
        for keyB in keys:
            if not keyB in valid_keys or keyA == keyB:
                continue
            print keyA + "--" + keyB
            filterB = getFilter(keyB)
            sub_dfA = data_df[filter_recent & filterA & ~filterB]
            sub_dfB = data_df[filter_recent & filterB & ~filterA]
            if is_valid(sub_dfA) and not is_valid(sub_dfB):
                valid_keys.remove(keyB)
            elif not is_valid(sub_dfA) and is_valid(sub_dfB):
                valid_keys.remove(keyA)
                break
            elif not is_valid(sub_dfA) and not is_valid(sub_dfB):
                if output_df["count"][keyA] > output_df["count"][keyB]:
                    print keyB
                    valid_keys.remove(keyB)
                elif output_df["count"][keyA] < output_df["count"][keyB]:
                    print keyA
                    valid_keys.remove(keyA)
                    break
                else:
                    if keyA.split("-") > keyB.split("-"):
                        valid_keys.remove(keyB)
                    elif keyA.split("-") < keyB.split("-"):
                        valid_keys.remove(keyA)
                        break
                    else:
                        if 
                        
    return valid_keys

In [64]:
def is_valid(sub_df):
    sub_df = sub_df.sort_values("minute")
    sub_df = sub_df.groupby("date").first()
    count = len(sub_df)
    mean = sub_df.change.mean()
    if mean < 1.015 or count < 2:
        return False 
    win_df = sub_df[sub_df["change"] > 1.0]
    win_ratio = float(len(win_df))/len(sub_df)
    if count < 10:
        return mean > 1.025 and win_ratio > 0.75
    else:
        return mean > 1.02 and win_ratio > 0.6

In [32]:
data_df = pd.read_csv("top2.csv",index_col=0, parse_dates=True)
data_df = data_df[data_df["change"] < 1.22]
data_df = data_df[~data_df["minute_low"].isnull()]


def low(day):
    return "minute_low" if day == 2 else "low" + str(day)
def yz(day):
    return (data_df[low(day)] == data_df["high_limit" + str(day)])
def notyz(day):
    return (data_df[low(day)] < data_df["high_limit" + str(day)])
def top(day):
    return (data_df["close" + str(day)] == data_df["high_limit" + str(day)])
def nottop(day):
    return (data_df["close" + str(day)] < data_df["high_limit" + str(day)])
def opentop(day):
    return (data_df["open" + str(day)] == data_df["high_limit" + str(day)])
def jump(day):
    return (data_df[low(day)] > data_df["high" + str(day - 1)]) 
def foot(day,degree = 1.0):
    return (data_df[low(day)] >= data_df["open" + str(day)] * degree)
def speedup(day):
    return (data_df["high" + str(day)] - data_df[low(day)] < data_df["high" + str(day - 1)] - data_df[low(day - 1)])
def recent(day,valid_data_df):
    return valid_data_df["recent"] == day


isnew = data_df["isnew"] == 1
small_capq = data_df["capq"] < 0.5
small_cap = data_df["circap"] < 200
minute = (data_df["minute"] < "11:00:00")
small_volume = (data_df["minute_volume"] < data_df["volume1"] * 0.5)

#前天/昨天/今天开盘涨停
opentop0 = opentop(0)
opentop1 = opentop(1)
opentop2 = opentop(2)

# 前天/昨天/今天是/不是一字板
yz0 = yz(0)
yz1 = yz(1)
yz2 = yz(2)

# 前天/昨天是/否涨停
top0 = top(0)
top1 = top(1)

# 前天/昨天/今天 光脚
foot0 = foot(0,0.995)
foot1 = foot(1,0.995)
foot2 = foot(2,0.995)

# 昨天/今天 跳高
jump1 = jump(1)
jump2 = jump(2)

#昨天/今天 加速
speedup1 = speedup(1)
speedup2 = speedup(2)

def __eval(filter_name):
    if filter_name[0] == "~":
        return ~eval(filter_name[1:])
    else:
        return eval(filter_name)

def combineFilters(filters):
    return reduce(lambda x, y: x & y, [__eval(filter_name) for filter_name in filters] + [~yz2])

def getFilter(filter_key):
    filter_names = filter_key.split("-")[:-1]
    the_filter = combineFilters(filter_names)
    return the_filter